In [5]:
import pandas as pd
import os

In [6]:
base_dir = os.getcwd()
data_folder = os.path.join(base_dir, "..", "data")
data_folder = os.path.abspath(data_folder)

excel_files = ["brl_data_raw.xlsx", "vnd_data_raw.xlsx", "cop_data_raw.xlsx", "idr_data_raw.xlsx", "hnl_data_raw.xlsx"]

for file in excel_files:
    file_path = os.path.join(data_folder, file)

    df = pd.read_excel(file_path, sheet_name="Monthly")

    row_filter_dates = df.iloc[:, 4] == "Scale"

    df_dates = df[row_filter_dates].iloc[:, 5:].reset_index(drop=True).T
    df_dates.columns = ["Date"]

    df = pd.read_excel(file_path, sheet_name="Monthly", skiprows=5)

    if file == "hnl_data_raw.xlsx":
        row_filter_monthly = df.iloc[:, 1] == "Domestic Currency per U.S. Dollar, End of Period"
    else:
        row_filter_monthly = df.iloc[:, 1] == "Domestic Currency per U.S. Dollar, Period Average"

    df_monthly = df[row_filter_monthly].iloc[:, 5:].reset_index(drop=True).T
    df_monthly.columns = ["Exchange Rate"]

    file_prefix = file.split('_')[0][:3]

    df_monthly.columns = [f"{file_prefix} Exchange Rate"]

    final_df = pd.concat([df_dates, df_monthly], axis=1)

    final_df = final_df[final_df["Date"].str.match(r"^\d{4}M\d{2}$", na=False)]

    final_df["Date"] = pd.to_datetime(final_df["Date"].str.replace("M", "-"), format="%Y-%m").dt.strftime("%m/01/%Y")

    clean_file_name = file.replace("_raw.xlsx", "_clean.csv")
    clean_file_path = os.path.join(data_folder, clean_file_name)
    final_df.to_csv(clean_file_path, index=False)